In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

In [4]:
!ls ../data

Future_Schedule_20220301-20220331.xlsx
Realized_Schedule_20210101-20220228.xlsx


In [5]:
data_path = Path("../", "data")
realized_schedule = "Realized_Schedule_20210101-20220228.xlsx"
future_schedule = "Future_Schedule_20220301-20220331.xlsx"
df_raw_realized = pd.read_excel(Path(data_path, realized_schedule))
df_raw_future = pd.read_excel(Path(data_path, future_schedule))

In [195]:
import datetime
def add_date_features(df_proc, time_col="ScheduleTime"):
    """Adds one-hot encoded time features from datetime column to df_proc"""

    time_series = df_proc[time_col]
    S = time_series.dt

    df_time = pd.DataFrame({"day":S.day, "month":S.month,
                            "is_month_start":S.is_month_start, "is_month_end":S.is_month_end,
                            "is_year_start":S.is_year_start, "is_year_end":S.is_year_end,
                            "is_quarter_start":S.is_quarter_start, "is_quarter_end":S.is_quarter_end
                            })

    # Year 2021, 2022
    df_year = pd.get_dummies(pd.Categorical(S.year)).astype(bool)
    df_year.columns = ["year_" + str(s) for s in list(df_year.columns.values)]

    # Weekday
    df_weekday = pd.get_dummies(pd.Categorical(S.dayofweek)).astype(bool)
    df_weekday.columns = ["dayofweek_" + str(s+1) for s in list(df_weekday.columns.values)]

    df_out = pd.concat([df_proc, df_year, df_weekday, df_time], axis=1)
    df_out.drop(time_col, axis=1)

    return df_out

def add_cat_features(df_proc, cat_columns):
    """Adds one-hot encoded columns from cat_columns to df_proc"""
    
    #df_cat = df_raw_realized[["Airline", "Destination", "AircraftType", "FlightType", "Sector"]]
    #include = ["Destination", "Sector", "FlightType", "AircraftType"]

    dfs = []
    for col in cat_columns:
        df = pd.get_dummies(pd.Categorical(df_proc[col]))
        df.columns = [col + "_" + str(s) for s in list(df.columns.values)]
        dfs.append(df)

    df_onehot = pd.concat(dfs, axis=1)
    df_out = pd.concat([df_proc, df_onehot], axis=1)
    df_out = df_out.drop(cat_cols, axis=1)

    return df_out

def normalize_minmax_cols(df_proc, norm_cols):
    """ Min-max normalizes norm_cols in df_proc """

    # Normalize
    for col in norm_cols:
        x = df_proc[col]
        x_norm = (x - x.min())
        x_norm = x_norm / x_norm.max()

        df_proc[col] = x_norm

    return df_proc

def add_time_delta(df_proc, end_time="2022-03-31", time_col="ScheduleTime"):
    """Adds column representing time proximity to end date in fractional months"""
    end_time = pd.to_datetime(end_time)

    df = df_proc[time_col]
    time_delta =  (df - end_time) / np.timedelta64(1, "M") * -1
    time_delta = 1/time_delta
    time_delta = (time_delta - time_delta.min())

    df_proc["time_delta"] = time_delta
    
    return df_proc

In [169]:
end_time = pd.to_datetime("2022-03-31")
df = df_raw_realized["ScheduleTime"].copy()
time_delta =  (df - end_time) / np.timedelta64(1, "M") * -1
time_delta = 1/time_delta

0        0.067082
1        0.067107
2        0.067116
3        0.067124
4        0.067130
           ...   
39444    1.007218
39445    1.008145
39446    1.008957
39447    1.008261
39448    1.010935
Name: ScheduleTime, Length: 39449, dtype: float64

In [225]:
df_proc = df_raw_realized.copy()

# Add feature how close to ending date range 0-1 (1 is end date)
df_proc = add_time_delta(df_proc, end_time="2022-03-31", time_col="ScheduleTime")

# Create one-hot encoded time features
df_proc = add_date_features(df_proc, "ScheduleTime")

# One-hot encode other features
cat_cols = ["Destination", "Sector", "FlightType", "AircraftType"]
df_proc = add_cat_features(df_proc, cat_cols)

# Exclude columns
exclude_cols = ["Airline", "FlightNumber"]
df_proc = df_proc.drop(exclude_cols, axis=1)

# Normalize
norm_cols = ["SeatCapacity"]
df_proc = normalize_minmax_cols(df_proc, norm_cols)


In [247]:
def create_trainval(df_proc, val_months, val_year="year_2021", exclude_cols=["ScheduleTime"], y_col="LoadFactor"):
    """ Creates training and validation datasets by selected months/years """

    # FloatingPointError
    m1 = df_proc[val_year] == True
    m2 = df_proc["month"].isin(val_months)
    mc = np.logical_and(m1, m2)

    val_idxs = df_proc[~mc].index
    train_idxs = df_proc[mc].index

    # Remove target y column and any other columns to exclude
    y_col = "LoadFactor"
    exclude_cols = [y_col] + exclude_cols
    print(f"Excluding cols in features: {exclude_cols}")

    dataset_X = df_proc.drop(exclude_cols, axis=1)
    dataset_y = df_proc[y_col]

    X_train, y_train = dataset_X.loc[train_idxs], dataset_y.loc[train_idxs]
    X_val, y_val = dataset_X.loc[val_idxs], dataset_y.loc[val_idxs]

    return X_train, y_train, X_val, y_val

In [248]:
val_months = [11]
val_year = "year_2021"
exclude_cols = ["ScheduleTime"]

X_train, y_train, X_val, y_val = create_trainval(df_proc, val_months, val_year, exclude_cols)

Excluding cols in features: ['LoadFactor', 'ScheduleTime']
